In [1]:
import datajoint as dj
dj.config["database.host"] = "db.datajoint.com"
dj.config["database.user"] = "microns"
dj.config["database.password"] = "microns2023"
nda = dj.create_virtual_module('nda', 'microns_phase3_nda')

import h5py
import numpy as np
from pynwb import NWBHDF5IO
# from h5py import File
# from fsspec.implementations.cached import CachingFileSystem

Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


In [2]:
# Source data file paths
scan_keys = list(nda.Scan.proj())
k = scan_keys[1]
display(k)

{'session': 4, 'scan_idx': 9}

In [3]:
# get local nwb file (newly generated with updated code)
ophys_file_version = 2
local_parent_path = "/mnt/at-export01/17797-export/DANDI_NWB/"
local_file_stem = f"functional_scan_17797_{k['session']}_{k['scan_idx']}_v{ophys_file_version}"
local_file_path = local_parent_path + local_file_stem + '/' + local_file_stem + ".nwb"
local_file = h5py.File(local_file_path,mode='r')
local_file_io = NWBHDF5IO(file=local_file,load_namespaces=True)
local_nwbfile = local_file_io.read()

# get dandi local nwb file (previously generated by catalyst, downlaoded from dandiarchive)
dandi_parent_path = '/mnt/at-export01/17797-export/DANDI_NWB/dandi_download_000402_2023_11_21/000402/sub-17797/'
dandi_file_path = dandi_parent_path + f"sub-17797_ses-{k['session']}-scan-{k['scan_idx']}_behavior+image+ophys.nwb"
dandi_file = h5py.File(dandi_file_path,mode='r')
dandi_file_io = NWBHDF5IO(file=dandi_file)#,load_namespaces=True)
dandi_nwbfile = dandi_file_io.read()

# # get dandi streaming nwb file (not recommended since big data pulls are slow on runtime)
# from dandi.dandiapi import DandiAPIClient
# from fsspec import filesystem
# dandiset_id = "000402"
# # file_path = "sub-17797/sub-17797_ses-9-scan-4_behavior+image+ophys.nwb" # file size ~67GB

# dandi_path = f"sub-17797/sub-17797_ses-{k['session']}-scan-{k['scan_idx']}_behavior+image+ophys.nwb"

# # Get the location of the file on DANDI
# with DandiAPIClient() as client:
#     asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(dandi_path)
#     s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

# # Create a virtual filesystem based on the http protocol and use caching to save accessed data to RAM.
# fs = filesystem("http")
# dandi_file_path = fs.open(s3_url, "rb")
# dandi_file = h5py.File(dandi_file_path, mode="r")
# # Open the file with NWBHDF5IO
# dandi_file_io = NWBHDF5IO(file=dandi_file, load_namespaces=True)
# dandi_nwbfile = dandi_file_io.read()

/usr/local/lib/python3.8/site-packages/pynwb/image.py:106: UserWarning: ImageSeries 'Video: stimulus_17797_4_9_v4': Length of data does not match length of timestamps. Your data may be transposed. Time should be on the 0th dimension
  warnings.warn(


In [4]:
def compare_nwb_files(file1_path, file2_path, verbose=False, excluded_attributes = set(['object_id']), excluded_datasets = set(['data','file_create_date','identifier','session_description'])):
    def compare_arrays(arr1, arr2, n_samples = 5, path="/"):
        """
        Compares the first, last, and random n elements of two numpy arrays.
        """
        indices_to_compare = sorted([0, len(arr1)-1] + list(np.random.choice(len(arr1), size=np.min((n_samples,len(arr1))), replace=False)))
    
        arr1_subset = arr1[indices_to_compare]
        arr2_subset = arr2[indices_to_compare]

        if arr1_subset.dtype == 'O' and arr2_subset.dtype == 'O':
            if not np.array_equal(arr1_subset,arr2_subset):
                print(f"Difference in array '{path}' (subset):")
                print(f"  Array 1: {arr1_subset}")
                print(f"  Array 2: {arr2_subset}")
            else:
                if verbose:
                    print(f"No difference in array '{path}' (subset):")
        else:
            if not np.array_equal(arr1_subset, arr2_subset,equal_nan=True):
                print(f"Difference in array '{path}' (subset):")
                print(f"  Array 1: {arr1_subset}")
                print(f"  Array 2: {arr2_subset}")
            else:
                if verbose:
                    print(f"No difference in array '{path}' (subset):")
    
    def compare_datasets(dataset1, dataset2, path="/"):
        """
        Recursively compares datasets within nested structures.
        """
        if isinstance(dataset1, h5py.Group) and isinstance(dataset2, h5py.Group):
            # Compare group attributes (excluding 'object_id')
            attrs1 = {key: dataset1.attrs[key] for key in dataset1.attrs if key not in excluded_attributes}
            # attrs2 = {key: dataset2.attrs[key] for key in dataset2.attrs if key != 'object_id'}
            attrs2 = {key: dataset2.attrs[key] for key in dataset2.attrs if key not in excluded_attributes}

            key_match = attrs1.keys() == attrs2.keys()
            non_reference_match = np.all([np.all(v1==v2) for v1,v2 in zip(attrs1.values(),attrs2.values()) 
                                          if not np.all([isinstance(v,h5py.h5r.Reference) for v in (v1,v2)])])
            reference_match = np.all([file1[v1].name == file2[v2].name for v1,v2 in zip(attrs1.values(),attrs2.values()) 
                                      if np.all([isinstance(v,h5py.h5r.Reference) for v in (v1,v2)])])

            if not (key_match and non_reference_match and reference_match):
            # if attrs1 != attrs2:
                print(f"Difference in attributes in group '{path}'")
                print(f"  attr 1: {attrs1}")
                print(f"  attr 2: {attrs2}")
            else:
                if verbose:
                    print(f"No difference in attributes in group '{path}'")

            # Recursively compare datasets in the group
            for key in dataset1.keys():
                if key in dataset2:
                    if key in excluded_datasets:
                        print(f'skipping dataset {key}')
                    else:
                        compare_datasets(dataset1[key], dataset2[key], path + "/" + key)
                else:
                    print(f"Dataset '{key}' is missing in '{path}' in second file")
            
            for key in dataset2.keys():
                if key not in dataset1:
                    print(f"Dataset '{key}' is missing in '{path}' in first file")

        elif isinstance(dataset1, h5py.Dataset) and isinstance(dataset2, h5py.Dataset):
            # Compare dataset attributes (excluding 'object_id')
            attrs1 = {key: dataset1.attrs[key] for key in dataset1.attrs if key != 'object_id'}
            attrs2 = {key: dataset2.attrs[key] for key in dataset2.attrs if key != 'object_id'}

            key_match = attrs1.keys() == attrs2.keys()
            non_reference_match = np.all([np.all(v1==v2) for v1,v2 in zip(attrs1.values(),attrs2.values()) 
                                          if not np.all([isinstance(v,h5py.h5r.Reference) for v in (v1,v2)])])
            reference_match = np.all([file1[v1].name == file2[v2].name for v1,v2 in zip(attrs1.values(),attrs2.values()) 
                                      if np.all([isinstance(v,h5py.h5r.Reference) for v in (v1,v2)])])

            if not (key_match and non_reference_match and reference_match):
            # if attrs1 != attrs2:
                print(f"Difference in attributes in dataset '{path}'")
                print(f"  attr 1: {attrs1}")
                print(f"  attr 2: {attrs2}")
            else:
                if verbose:
                    print(f"No difference in attributes in dataset '{path}'")

            # Compare dataset shapes
            if dataset1.shape != dataset2.shape:
                print(f"Difference in shapes in dataset '{path}': {dataset1.shape} vs {dataset2.shape}")
                return

            # Compare dataset values (including arrays)
            if isinstance(dataset1[()], np.ndarray) and isinstance(dataset2[()], np.ndarray):# and len(dataset1[()])>2 and len(dataset2[()])>2:
                compare_arrays(dataset1[()], dataset2[()], path=path)
            elif not np.all((dataset1[()] == dataset2[()])):
                if dataset1.dtype == 'O' and dataset2.dtype == 'O':
                    print(f"Difference in dataset '{path}':")
                    print(f"  File 1: {dataset1[()]}")
                    print(f"  File 2: {dataset2[()]}")
                elif not np.all((dataset1[()] == dataset2[()]) | (np.isnan(dataset1[()]) & np.isnan(dataset2[()]))):
                    print(f"Difference in dataset '{path}':")
                    print(f"  File 1: {dataset1[()]}")
                    print(f"  File 2: {dataset2[()]}")
                else:
                    if verbose:
                        print(f"No difference in dataset '{path}':")
            else:
                if verbose:
                    print(f"No difference in dataset '{path}':")
        else:
            print(f"unhandled type {type(dataset1)},{type(dataset2)}")
            

    # Open the NWB files
    with h5py.File(file1_path, 'r') as file1, h5py.File(file2_path, 'r') as file2:
        # Compare the root groups
        compare_datasets(file1, file2)

In [5]:
compare_nwb_files(local_file_path,dandi_file_path,excluded_datasets = set(['data']))

Difference in attributes in group '/'
  attr 1: {'.specloc': <HDF5 object reference>, 'namespace': 'core', 'neurodata_type': 'NWBFile', 'nwb_version': '2.6.0'}
  attr 2: {'.specloc': <HDF5 object reference>, 'namespace': 'core', 'neurodata_type': 'NWBFile', 'nwb_version': '2.5.0'}
skipping dataset data
skipping dataset data
skipping dataset data
skipping dataset data
skipping dataset data
skipping dataset data
skipping dataset data
skipping dataset data
skipping dataset data
skipping dataset data
skipping dataset data
skipping dataset data
Difference in shapes in dataset '//acquisition/Video: stimulus_17797_4_9_v4/timestamps': (334501,) vs (334509,)
skipping dataset data
Difference in array '//file_create_date' (subset):
  Array 1: [b'2023-11-20T20:47:40.786061+00:00' b'2023-11-20T20:47:40.786061+00:00'
 b'2023-11-20T20:47:40.786061+00:00']
  Array 2: [b'2023-02-12T21:42:50.494421+00:00' b'2023-02-12T21:42:50.494421+00:00'
 b'2023-02-12T21:42:50.494421+00:00']
Difference in attributes 

In [ ]:
file1 = h5py.File(local_file_path,'r')
date1 = file1['file_create_date']

file2 = h5py.File(dandi_file_path,'r')
date2 = file2['file_create_date']

In [145]:
attrs1['table']

<HDF5 object reference>

In [6]:
%pip show pynwb

Name: pynwb
Version: 2.5.0
Summary: Package for working with Neurodata stored in the NWB format
Home-page: https://github.com/NeurodataWithoutBorders/pynwb
Author: Andrew Tritt
Author-email: ajtritt@lbl.gov
License: BSD
Location: /usr/local/lib/python3.8/site-packages
Requires: h5py, hdmf, numpy, pandas, python-dateutil, setuptools
Required-by: dandi, microns-to-nwb, neuroconv, nwbinspector, roiextractors
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip show neuroconv

Name: neuroconv
Version: 0.2.4
Summary: Convert data from proprietary formats to NWB format.
Home-page: https://github.com/catalystneuro/neuroconv
Author: Cody Baker, Szonja Weigl, Heberto Mayorquin, Luiz Tauffer, and Ben Dichter.
Author-email: ben.dichter@catalystneuro.com
License: UNKNOWN
Location: /usr/local/lib/python3.8/site-packages
Requires: dandi, h5py, hdmf, jsonschema, numpy, pandas, psutil, pynwb, PyYAML, scipy, tqdm
Required-by: microns-to-nwb
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip show roiextractors

Name: roiextractors
Version: 0.5.4
Summary: Python module for extracting optical physiology ROIs and traces for various file types and formats
Home-page: https://github.com/catalystneuro/roiextractors
Author: Heberto Mayorquin, Szonja Weigl, Cody Baker, Ben Dichter, Alessio Buccino
Author-email: ben.dichter@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.8/site-packages
Requires: dill, h5py, lazy-ops, psutil, pynwb, PyYAML, scipy, tqdm
Required-by: microns-to-nwb
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip show tifffile

Name: tifffile
Version: 2023.3.21
Summary: Read and write TIFF files
Home-page: https://www.cgohlke.com
Author: Christoph Gohlke
Author-email: cgohlke@cgohlke.com
License: BSD
Location: /usr/local/lib/python3.8/site-packages
Requires: numpy
Required-by: microns-to-nwb
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip show opencv-python-headless

Name: opencv-python-headless
Version: 4.8.1.78
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/opencv/opencv-python
Author: 
Author-email: 
License: Apache 2.0
Location: /usr/local/lib/python3.8/site-packages
Requires: numpy, numpy
Required-by: microns-to-nwb
Note: you may need to restart the kernel to use updated packages.


In [132]:
attrs1 = {key: file1.attrs[key] for key in file1.attrs if key not in excluded_attributes}
attrs2 = {key: file2.attrs[key] for key in file2.attrs if key not in excluded_attributes}
display(attrs1)
display(attrs2)

{'.specloc': <HDF5 object reference>,
 'namespace': 'core',
 'neurodata_type': 'NWBFile',
 'nwb_version': '2.6.0'}

{'.specloc': <HDF5 object reference>,
 'namespace': 'core',
 'neurodata_type': 'NWBFile',
 'nwb_version': '2.5.0'}

In [133]:
attrs1 = {key: file1['general']['subject']['age'].attrs[key] for key in file1['general']['subject']['age'].attrs if key not in excluded_attributes}
attrs2 = {key: file2['general']['subject']['age'].attrs[key] for key in file2['general']['subject']['age'].attrs if key not in excluded_attributes}
display(attrs1)
display(attrs2)

{'reference': 'birth'}

{}

In [147]:
attrs1 = {key: file1['processing']['ophys']['Fluorescence']['RoiResponseSeries1']['rois'].attrs[key] for key in file1['processing']['ophys']['Fluorescence']['RoiResponseSeries1']['rois'].attrs if key not in excluded_attributes}
attrs2 = {key: file2['processing']['ophys']['Fluorescence']['RoiResponseSeries1']['rois'].attrs[key] for key in file2['processing']['ophys']['Fluorescence']['RoiResponseSeries1']['rois'].attrs if key not in excluded_attributes}
display(attrs1)
display(attrs2)

{'description': 'all rois in field 1',
 'namespace': 'hdmf-common',
 'neurodata_type': 'DynamicTableRegion',
 'table': <HDF5 object reference>}

{'description': 'all rois in field 1',
 'namespace': 'hdmf-common',
 'neurodata_type': 'DynamicTableRegion',
 'table': <HDF5 object reference>}

In [177]:
dataset1 = file1['processing']['ophys']['Fluorescence']['RoiResponseSeries1']['rois']
dataset2 = file2['processing']['ophys']['Fluorescence']['RoiResponseSeries1']['rois']

In [181]:
attrs1

{'description': 'all rois in field 1',
 'namespace': 'hdmf-common',
 'neurodata_type': 'DynamicTableRegion',
 'table': <HDF5 object reference>}

In [182]:
attrs2

{'description': 'all rois in field 1',
 'namespace': 'hdmf-common',
 'neurodata_type': 'DynamicTableRegion',
 'table': <HDF5 object reference>}

In [183]:
attrs1.keys() == attrs2.keys()

True

In [184]:
np.all([np.all(v1==v2) for v1,v2 in zip(attrs1.values(),attrs2.values())])

False

In [199]:
if isinstance(dataset1, h5py.Dataset) and isinstance(dataset2, h5py.Dataset):
    # Compare group attributes (excluding 'object_id')
    attrs1 = {key: dataset1.attrs[key] for key in dataset1.attrs if key not in excluded_attributes}
    # attrs2 = {key: dataset2.attrs[key] for key in dataset2.attrs if key != 'object_id'}
    attrs2 = {key: dataset2.attrs[key] for key in dataset2.attrs if key not in excluded_attributes}

    if not (attrs1.keys() == attrs2.keys() and np.all([np.all(v1==v2) for v1,v2 in zip(attrs1.values(),attrs2.values())])):
    # if attrs1 != attrs2:
        print(f"Difference in attributes in group '{path}'")
        print(f"  attr 1: {attrs1}")
        print(f"  attr 2: {attrs2}")
    else:
        if verbose:
            print(f"No difference in attributes in group '{path}'")

Difference in attributes in group 'temp'
  attr 1: {'description': 'all rois in field 1', 'namespace': 'hdmf-common', 'neurodata_type': 'DynamicTableRegion', 'table': <HDF5 object reference>}
  attr 2: {'description': 'all rois in field 1', 'namespace': 'hdmf-common', 'neurodata_type': 'DynamicTableRegion', 'table': <HDF5 object reference>}


In [225]:
v1 = list(attrs1.values())[-1]
v2 = list(attrs2.values())[-1]

In [232]:
file1[v1].name == file2[v2].name

'/processing/ophys/ImageSegmentation/PlaneSegmentation1'

In [227]:
dir(v1)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'typecode',
 'typesize']

In [209]:
v1

<HDF5 object reference>

In [212]:
type(v2)

h5py.h5r.Reference

In [213]:
isinstance(v2,h5py.h5r.Reference)

True

In [201]:
attrs1.values()

dict_values(['all rois in field 1', 'hdmf-common', 'DynamicTableRegion', <HDF5 object reference>])

In [188]:
file1[v1].name == file2[v2].name if np.all((v1.

<HDF5 dataset "rois": shape (643,), type "<i8">

In [172]:
file1[attrs1['table']].name == file2[attrs2['table']].name

True

In [169]:
dir(file1[attrs1['table']])

['_MutableMapping__marker',
 '__abstractmethods__',
 '__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_d',
 '_e',
 '_gcpl_crt_order',
 '_id',
 '_ipython_key_completions_',
 '_lapl',
 '_lcpl',
 'attrs',
 'build_virtual_dataset',
 'clear',
 'copy',
 'create_dataset',
 'create_dataset_like',
 'create_group',
 'create_virtual_dataset',
 'file',
 'get',
 'id',
 'items',
 'keys',
 'move',
 'name',
 'parent',
 'pop',
 'popitem',
 'ref',
 'regionref',
 'require_dataset',
 'require_

In [167]:
display(file1[attrs1['table']])
display(file2[attrs2['table']])

<HDF5 group "/processing/ophys/ImageSegmentation/PlaneSegmentation1" (5 members)>

<HDF5 group "/processing/ophys/ImageSegmentation/PlaneSegmentation1" (5 members)>

In [ ]:
def are_object_references_equal(file1, ref1, file2, ref2):
    # Get the objects pointed to by the references
    obj1 = file1[ref1]
    obj2 = file2[ref2]

    # Compare the paths of the objects
    path1 = file1.get(obj1.name, None)
    path2 = file2.get(obj2.name, None)

    return path1 == path2

In [150]:
fluorescence_traces1 = file1['processing']['ophys']['Fluorescence']['RoiResponseSeries1']['data'][()]


In [152]:
np.shape(fluorescence_traces1)

(40000, 643)

In [164]:
roi_table1 = file1['processing']['ophys']['Fluorescence']['RoiResponseSeries6']['rois'][()]
roi_table1

array([   0,    1,    2, ..., 1408, 1409, 1410])

In [137]:
attrs1 = {key: file1['processing']['ophys']['ImageSegmentation']['PlaneSegmentation1'].attrs[key] for key in file1['processing']['ophys']['ImageSegmentation']['PlaneSegmentation1'].attrs if key not in excluded_attributes}
attrs2 = {key: file2['processing']['ophys']['ImageSegmentation']['PlaneSegmentation1'].attrs[key] for key in file2['processing']['ophys']['ImageSegmentation']['PlaneSegmentation1'].attrs if key not in excluded_attributes}
display(attrs1)
display(attrs2)

{'colnames': array(['image_mask', 'mask_type'], dtype=object),
 'description': 'The output from segmenting field 1 contains the image masks (weights and mask classification) and the structural ids extracted from the CAVE database on 2023-12-04. To access the latest revision from the live resource see the notebook that is linked to the dandiset. The structual ids might not exist for all plane segmentations.',
 'namespace': 'core',
 'neurodata_type': 'PlaneSegmentation'}

{'colnames': array(['image_mask', 'mask_type'], dtype=object),
 'description': 'The output from segmenting field 1 contains the image masks (weights and mask classification) and the structural ids extracted from the CAVE database on 2023-02-12. To access the latest revision from the live resource see the notebook that is linked to the dandiset. The structual ids might not exist for all plane segmentations.',
 'namespace': 'core',
 'neurodata_type': 'PlaneSegmentation'}

In [139]:
attrs1 = {key: file1['processing']['ophys']['ImageSegmentation']['PlaneSegmentation2']['cave_ids_index'].attrs[key] for key in file1['processing']['ophys']['ImageSegmentation']['PlaneSegmentation2']['cave_ids_index'].attrs if key not in excluded_attributes}
attrs2 = {key: file2['processing']['ophys']['ImageSegmentation']['PlaneSegmentation2']['cave_ids_index'].attrs[key] for key in file2['processing']['ophys']['ImageSegmentation']['PlaneSegmentation2']['cave_ids_index'].attrs if key not in excluded_attributes}
display(attrs1)
display(attrs2)

{'description': "Index for VectorData 'cave_ids'",
 'namespace': 'hdmf-common',
 'neurodata_type': 'VectorIndex',
 'target': <HDF5 object reference>}

{'description': "Index for VectorData 'cave_ids'",
 'namespace': 'hdmf-common',
 'neurodata_type': 'VectorIndex',
 'target': <HDF5 object reference>}

In [ ]:
# TODO: optional argument to skip testing large arrays
# TODO add verbose option and/or logging
# TODO find a way to build the files with NWB_version 2.5.0 instead of 2.6.0
# TODO find a way to build with hdmf-common version 1.5.1 instead of 1.8.0
# TODO find a way to build with hdmf-experimental version 0.2.0 instead of 0.5.0
# TODO print only the part of the attributes that is different, not the whole thing

In [26]:
np.isnan(imaging_rate1[()]) and np.isnan(imaging_rate2[()])

True

In [27]:
arr1 = imaging_rate1[()]

In [28]:
arr2 = imaging_rate2[()]

In [30]:
arr1==arr2

False

In [31]:
np.isnan(arr1) & np.isnan(arr2)

True

In [35]:
arr1 = np.array([1, np.nan])

In [41]:
arr2 = np.array([1, np.nan])

In [39]:
((arr1 == arr2) | (np.isnan(arr1) & np.isnan(arr2)))

array([False, False])

In [29]:
((arr1 == arr2) | (np.isnan(arr1) & np.isnan(arr2))).all()

True

In [22]:
file1['file_create_date'][:].dtype

dtype('O')

In [28]:
file1['acquisition']['EyeTracking']['eye_position']['data'][:].dtype

dtype('float64')

In [58]:
file1['acquisition']['EyeTracking']['eye_position']['reference_frame'][()] == file2['acquisition']['EyeTracking']['eye_position']['reference_frame'][()]


True

In [59]:
dataset1 = file1['acquisition']['EyeTracking']['eye_position']['reference_frame']
dataset2 = file2['acquisition']['EyeTracking']['eye_position']['reference_frame']

In [57]:
file2['acquisition']['EyeTracking']['eye_position']['reference_frame'][()]

b'unknown'

In [50]:
file1['acquisition']['EyeTracking']['eye_position']['reference_frame']

<HDF5 dataset "reference_frame": shape (), type "|O">

In [42]:
file1['acquisition']['TwoPhotonSeries1']['imaging_plane']['imaging_rate'].dtype

dtype('<f8')

In [ ]:
# Attributes differ in group '/'
# Shapes differ in dataset '//acquisition/Video: stimulus_17797_4_9_v4/timestamps': (334501,) vs (334509,)